## 13. Resume training from checkpoint  
Ray automatically resumes from the latest checkpoint if one is available. This makes fault tolerance seamless—if the system interrupts a node or it fails, training can pick up where it left off with no manual intervention.

In [ ]:
# 13. Demonstrate fault-tolerant resume
result = trainer.fit()
print("Metrics after resume run:", result.metrics)